In [ ]:
import pandas as pd
import numpy as np
import matplotlib
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train = pd.read_csv("train_ready_for_model.csv")
train.head()

In [ ]:
test = pd.read_csv("test_ready_for_model.csv")
test.head()

In [188]:
result = test[["Outlet_Identifier","Item_Identifier"]]
result.head()

,Outlet_Identifier,Item_Identifier
0,OUT049,FDW58
1,OUT017,FDW14
2,OUT010,NCN55
3,OUT017,FDQ58
4,OUT027,FDY38


In [189]:
train.columns

Index(['Item_Fat_Content_Low Fat', 'Item_Fat_Content_Regular',
       'Item_Type_Baking Goods', 'Item_Type_Breads', 'Item_Type_Breakfast',
       'Item_Type_Canned', 'Item_Type_Dairy', 'Item_Type_Frozen Foods',
       'Item_Type_Fruits and Vegetables', 'Item_Type_Hard Drinks',
       ...
       'Item_MRP_Outlet_Type_Supermarket Type3_Outlet_Type_Supermarket Type3_Outlet_Identifier_OUT027_Item_MRP_Outlet_Type_Supermarket Type3_Outlet_Size_Medium_Outlet_Identifier_OUT027',
       'Item_MRP_Outlet_Type_Supermarket Type3_Outlet_Type_Supermarket Type3_Outlet_Identifier_OUT027_Item_MRP_Outlet_Identifier_OUT027_Outlet_Type_Supermarket Type3_Outlet_Size_Medium',
       'Item_MRP_Outlet_Type_Supermarket Type3_Outlet_Type_Supermarket Type3_Outlet_Identifier_OUT027_Item_MRP_Outlet_Identifier_OUT027_Outlet_Type_Supermarket Type3_Outlet_Identifier_OUT027',
       'Item_MRP_Outlet_Type_Supermarket Type3_Outlet_Type_Supermarket Type3_Outlet_Identifier_OUT027_Item_MRP_Outlet_Identifier_OUT027_Outlet_S

target = 'Item_Outlet_Sales'
IDcol = ['Item_Identifier','Outlet_Identifier']
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn import model_selection, metrics
from boruta import BorutaPy
predictors = [x for x in train.columns if x not in [target]+IDcol]
X=train[predictors].values
y=train[target].values
y=y.ravel()
rf =  LinearRegression()
feat_selector = BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)
feat_selector.fit(X,y)
feat_selector.support_


print(np.where(feat_selector.support_))
train[['Outlet_Identifier_OUT010'
'Outlet_Location_Type_Tier 3',
'Outlet_Type_Supermarket Type2',
'Item_Identifier_Pattern_NC',
'Outlet_Type_Pattern_Grocery',
'Outlet_Type_Pattern_SuperMarket',
'Item_Weight']].head()

feat_selector.ranking_

In [190]:
#Define target and ID columns:
target = 'Item_Outlet_Sales'
IDcol = ['Item_Identifier','Outlet_Identifier']
from sklearn import model_selection, metrics

def modelfit(alg, dtrain, dtest, predictors, target, IDcol, filename):
    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain[target])
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain[predictors])

    #Perform cross-validation:
    #cv_train_score = model_selection.cross_val_score(alg, dtrain[predictors], dtrain[target], cv=20, scoring='neg_mean_squared_error')
    #cv_train_score = np.sqrt(np.abs(cv_train_score))
    
    #Print model report:
    print("\nModel Report")
    #print("Train CV score : ", cv_train_score)
    print("Train RMSE : %.4g" % np.sqrt(metrics.mean_squared_error(dtrain[target].values, dtrain_predictions)))
    
    #Predict on testing data:
    dtest[target] = alg.predict(dtest[predictors])
    
    #Export submission file:
    IDcol.append(target)
    submission = pd.DataFrame({ x: dtest[x] for x in IDcol})
    submission.to_csv(filename, index=False)

In [191]:
from sklearn.linear_model import LinearRegression
predictors = [x for x in train.columns if x not in [target]+IDcol]
#print("predictors : ",predictors)
# print predictors
alg1 = LinearRegression(normalize=False)
modelfit(alg1, train, test, predictors, target, IDcol, 'res2.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#print(coef1)
#coef1.plot(kind='bar', title='Model Coefficients')


Model Report
Train RMSE : 1067


In [194]:
from sklearn.linear_model import Ridge
predictors = [x for x in train.columns if x not in [target]+IDcol]
#print("predictors : ",predictors)
# print predictors
alg1 = Ridge(alpha=0.65, solver='auto',normalize=False)
#Ridge(alpha = 1.0,normalize=)
modelfit(alg1, train, test, predictors, target, IDcol, 'res3.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')


Model Report
Train RMSE : 1067


In [60]:
from sklearn.linear_model import Lasso
#predictors = [x for x in train.columns if x not in [target]+IDcol]
# print predictors
alg1 = Lasso(alpha=.5
             ,    normalize=False)
#Lasso(alpha = 1)
modelfit(alg1, train, test, predictors, target, IDcol, 'res4.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')


Model Report
Train RMSE : 1069


In [173]:
from sklearn.linear_model import ElasticNet
predictors = [x for x in train.columns if x not in [target]+IDcol]
# print predictors
alg1 = ElasticNet(alpha=0.6,l1_ratio=0.4,normalize=False,selection= 'random')
#ElasticNet(alpha=0.01,l1_ratio=0.5,normalize=True)
modelfit(alg1, train, test, predictors, target, IDcol, 'res5.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')


Model Report
Train CV score :  [1146.89227504 1046.89629022 1070.7626526  1088.00364683 1092.77212548
 1060.82917643 1030.60402888 1070.73127246 1092.29857507 1069.58834728
 1015.12495228 1090.62870685 1075.08051842 1024.83889979 1009.23750308
 1147.13502858 1109.46500903 1006.54017178 1149.52547509 1039.94984195]
Train RMSE : 1069


In [12]:
from sklearn.neighbors import KNeighborsRegressor

In [13]:
predictors = [x for x in train.columns if x not in [target]+IDcol]
# print predictors
for i in range(1,20):
    alg1 = KNeighborsRegressor(n_neighbors=i)
    modelfit(alg1, train, test, predictors, target, IDcol, 'res6.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')


Model Report
Train CV score :  [2320.91726291 2322.96027584 2061.1738995  1760.88682998 1676.47073596
 1630.42298918 1609.44802353 1482.51773668 1573.46695769 1445.24719286
 1504.70307406 1577.62768705 1667.18146267 1322.92246894 1581.90731645
 1596.66053537 1629.21892572 1423.92706911 1558.66108226 1613.59990255]
Train RMSE : 0

Model Report
Train CV score :  [2283.60821131 2092.87505125 1770.4450987  1554.09301891 1522.84869218
 1373.89369981 1370.96097368 1390.32691661 1363.76848304 1257.33944097
 1309.44476501 1380.2063592  1478.89295789 1256.18155838 1359.15579171
 1411.14559337 1473.99577676 1300.29457244 1407.76953467 1441.73098291]
Train RMSE : 806.4

Model Report
Train CV score :  [2244.98032094 1959.72542316 1693.93715525 1564.79068495 1467.20032461
 1354.24454978 1352.11502422 1371.96858562 1303.7371964  1228.34385779
 1283.72353266 1271.97554223 1392.37270439 1199.9561323  1252.45446233
 1385.66947032 1411.69304341 1253.9301169  1388.76723829 1363.48973721]
Train RMSE : 94

In [14]:
predictors = [x for x in train.columns if x not in [target]+IDcol]
# print predictors
#for i in range(1,20):
alg1 = KNeighborsRegressor(n_neighbors=6)
modelfit(alg1, train, test, predictors, target, IDcol, 'res6.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')


Model Report
Train CV score :  [2244.87740151 1881.75678475 1675.75142668 1549.68729081 1492.66903049
 1347.24985357 1317.004347   1425.96092106 1271.94609426 1224.17372072
 1267.34711378 1213.04395659 1328.48821305 1185.96899731 1240.16242475
 1378.2422822  1408.16644728 1233.39068355 1390.98563798 1339.24406825]
Train RMSE : 1121


In [102]:
from sklearn.tree import DecisionTreeRegressor

In [103]:
from sklearn.tree import DecisionTreeRegressor
#DecisionTreeRegressor
predictors = [x for x in train.columns if x not in [target]+IDcol]
# print predictors
alg1 = DecisionTreeRegressor(random_state=0)#DecisionTreeRegressor(max_depth=5, min_samples_split=2,min_samples_leaf=1)
modelfit(alg1, train, test, predictors, target, IDcol, 'res7.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')


Model Report
Train CV score :  [1482.77990655 1595.56389693 1530.70582011 1654.24996353 1550.31590728
 1583.37812298 1426.64903619 1627.24608446 1485.92114907 1541.51591227
 1552.20590206 1571.83078299 1486.1766721  1489.6367735  1432.22939648
 1531.38945514 1593.78634167 1347.03758002 1561.8048931  1548.56001581]
Train RMSE : 8.532e-15


In [184]:
predictors=['Outlet_Type_Pattern_SuperMarket_Item_MRP',
 'Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Item_MRP',
 'Item_MRP_Outlet_Type_Pattern_SuperMarket_Item_MRP',
 'Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Item_MRP',
 'Outlet_Type_Pattern_SuperMarket_Item_MRP_Outlet_Type_Pattern_SuperMarket_Item_MRP',
 'Item_MRP_Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Item_MRP',
 'Outlet_Type_Pattern_SuperMarket_Item_MRP_Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Item_MRP',
 'Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Item_MRP',
 'Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Item_MRP_Outlet_Type_Pattern_SuperMarket_Item_MRP',
 'Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Item_MRP_Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Item_MRP',
 'Outlet_Type_Pattern_SuperMarket_Item_MRP_Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Item_MRP',
 'Outlet_Type_Pattern_SuperMarket_Item_MRP_Outlet_Type_Pattern_SuperMarket_Item_MRP_Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Item_MRP',
 'Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Item_MRP_Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Item_MRP',
 'Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Item_MRP_Outlet_Type_Pattern_SuperMarket_Item_MRP_Outlet_Type_Pattern_SuperMarket_Item_MRP',
 'Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Item_MRP_Item_MRP_Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Item_MRP',
 'Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Item_MRP_Outlet_Type_Pattern_SuperMarket_Item_MRP_Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Item_MRP',
 'Item_MRP_Outlet_Type_Pattern_SuperMarket_Item_MRP_Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Item_MRP',
 'Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Item_MRP_Outlet_Type_Pattern_SuperMarket_Item_MRP_Outlet_Type_Pattern_SuperMarket_Item_MRP',
 'Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Item_MRP_Item_MRP_Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Item_MRP',
 'Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Item_MRP_Outlet_Type_Pattern_SuperMarket_Item_MRP_Outlet_Type_Pattern_SuperMarket_Outlet_Type_Pattern_SuperMarket_Item_MRP']

In [151]:
from sklearn.ensemble import RandomForestRegressor
#predictors = [x for x in train.columns if x not in [target]+IDcol]
# print predictors
alg1 = RandomForestRegressor(n_estimators=1000,max_depth=5, min_samples_split=10,min_samples_leaf=5)
#alg1 = RandomForestRegressor(n_estimators=80,max_depth=15)
#alg1 = RandomForestRegressor(n_estimators=500,max_depth=10, min_samples_split=100,min_samples_leaf=50)
modelfit(alg1, train, test, predictors, target, IDcol, 'res8.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')


Model Report
Train CV score :  [1170.62590036 1058.4062642  1086.80624259 1107.04323557 1108.94231643
 1079.11495788 1044.44174188 1095.30311602 1106.44068571 1070.74645329
 1036.85089773 1108.6450441  1083.04261844 1042.38854709 1011.45557219
 1163.5645116  1123.79258539 1015.00816591 1166.80658946 1064.91551364]
Train RMSE : 1063


In [92]:
from sklearn.ensemble import AdaBoostRegressor

In [118]:
#predictors = [x for x in train.columns if x not in [target]+IDcol]
#print predictors
alg1 = AdaBoostRegressor(n_estimators=5 )
modelfit(alg1, train, test, predictors, target, IDcol, 'res9.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')


Model Report
Train RMSE : 1099


In [119]:
from sklearn.ensemble import GradientBoostingRegressor

In [135]:
predictors = [x for x in train.columns if x not in [target]+IDcol]
# print predictors
#GradientBoostingRegressor(alpha=0.3,n_estimators=100)
alg1 = GradientBoostingRegressor(alpha=0.3, criterion='friedman_mse', init=None,
             learning_rate=0.2, loss='ls', max_depth=7, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=50, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)
modelfit(alg1, train, test, predictors, target, IDcol, 'res9.csv')
#coef1 = pd.Series(alg1.coef_, predictors).sort_values()
#coef1.plot(kind='bar', title='Model Coefficients')


Model Report
Train RMSE : 485.9


In [ ]:
from sklearn.ensemble import 

In [121]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV

param_dist = {
 'n_estimators': [50,60,70,80,85,90,95,100],
 'alpha':[0.2,0.3,0.4,0.5,0.6,0.7]
 }

pre_gs_inst = RandomizedSearchCV(GradientBoostingRegressor(),
 param_distributions = param_dist,
 cv=3,
 n_iter = 47,
 n_jobs=-1)

pre_gs_inst.fit(train[predictors], train[target])

RandomizedSearchCV(cv=3, error_score='raise',
          estimator=GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=47, n_jobs=-1,
          param_distributions={'n_estimators': [50, 60, 70, 80, 85, 90, 95, 100], 'alpha': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [122]:
pre_gs_inst.best_estimator_

GradientBoostingRegressor(alpha=0.4, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=50, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)